#Download and install the necessary dependencies

In [5]:
!pip install -q transformers==3.5.0 fastapi nest-asyncio pyngrok uvicorn
!pip install -U gdown
!pip install torch==1.4.0

Requirement already up-to-date: gdown in /usr/local/lib/python3.7/dist-packages (3.13.0)


In [6]:
! git clone -q https://github.com/microsoft/CodeXGLUE.git

fatal: destination path 'CodeXGLUE' already exists and is not an empty directory.


In [7]:
cd ./CodeXGLUE/Code-Text/code-to-text/code

/content/CodeXGLUE/Code-Text/code-to-text/code


In [8]:
import gdown

gdown.cached_download(
    "https://drive.google.com/uc?id=1CWcX7oT5xizjRpAwb6NcqHv4I1gQsaH8",
    "code_commenter_models.zip",
    postprocess=gdown.extractall,
)

File exists: code_commenter_models.zip


'code_commenter_models.zip'

In [9]:
%%capture
import torch

import torch.nn as nn

from model import Seq2Seq
from pathlib import Path

In [10]:
from transformers import AutoTokenizer

In [11]:
from transformers import RobertaConfig, RobertaModel

In [13]:
source_length = 256
target_length = 64
beam_size = 10
output_dir = f"model/java"
pretrained_model = "microsoft/codebert-base"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
config = RobertaConfig.from_pretrained(pretrained_model)
encoder = RobertaModel.from_pretrained(pretrained_model, config = config)    
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model = Seq2Seq(encoder = encoder,decoder = decoder,config=config,
                beam_size=beam_size,max_length=target_length,
                sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)


If you don't have gpu then use map_location=torch.device('cpu') with load model.load()

my_model = net.load_state_dict(torch.load('classifier.pt', map_location=torch.device('cpu')))

In [14]:
model.load_state_dict(torch.load(Path(output_dir)/"checkpoint-best-bleu/pytorch_model.bin"))
model.to("cuda")

Seq2Seq(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [15]:
from run import convert_examples_to_features, Example

class Args:
    max_source_length = source_length
    max_target_length = target_length

args = Args()

def prep_input(method):
    examples = [
        Example(0, source = method, target = "")
    ]
    eval_features = convert_examples_to_features(
        examples, tokenizer, args, stage="test"
    )
    source_ids = torch.tensor(
        eval_features[0].source_ids, dtype = torch.long
    ).unsqueeze(0).to("cuda")
    source_mask = torch.tensor(
        eval_features[0].source_mask, dtype = torch.long
    ).unsqueeze(0).to("cuda")

    return source_ids, source_mask

def gen_comment(method):
    source_ids, source_mask = prep_input(method)
    with torch.no_grad():
        pred = model(source_ids = source_ids, source_mask = source_mask)[0]
        t = pred[0].cpu().numpy()
        t = list(t)
        if 0 in t:
            t = t[:t.index(0)]
        text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
        return text

In [16]:
gen_comment("public static void main(){}")

'main method.'

In [ ]:
# colapse
# Adapted from from https://stackoverflow.com/questions/63833593/how-to-run-fastapi-uvicorn-in-google-colab
import nest_asyncio
import uvicorn

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok

class Message(BaseModel):
    code: str

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.post('/')
async def root(message: Message):
    cmt = gen_comment(message.code)
    return cmt

url = ngrok.connect(port=80)
print('Public URL:', url)
nest_asyncio.apply()
uvicorn.run(app, port=80)

05/29/2021 10:04:19 - INFO - pyngrok.ngrok -   Opening tunnel named: http-80-c351bf4f-13e7-4744-9a3c-286675a3758b


05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg="no configuration paths supplied"
05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg="tunnel session started" obj=tunnels.session
05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg="client session established" obj=csess id=acc2174009f7
05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg=start pg=/api/tunnel

Public URL: NgrokTunnel: "http://f2d605073198.ngrok.io" -> "http://localhost:80"


05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg=start pg="/api/tunnels/http-80-c351bf4f-13e7-4744-9a3c-286675a3758b (http)" id=e41c37a9b3bc5f8c
05/29/2021 10:04:21 - INFO - pyngrok.process.ngrok -   t=2021-05-29T10:04:21+0000 lvl=info msg=end pg="/api/tunnels/http-80-c351bf4f-13e7-4744-9a3c-286675a3758b (http)" id=e41c37a9b3bc5f8c status=200 dur=141.716µs
INFO:     Started server process [371]
05/29/2021 10:04:21 - INFO - uvicorn.error -   Started server process [371]
INFO:     Waiting for application startup.
05/29/2021 10:04:21 - INFO - uvicorn.error -   Waiting for application startup.
INFO:     Application startup complete.
05/29/2021 10:04:21 - INFO - uvicorn.error -   Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:80 (Press CTRL+C to quit)
05/29/2021 10:04:21 - INFO - uvicorn.error -   Uvicorn running on http://127.0.0.1:80 (Press CTRL+C to quit)
05/29/2021 10:11:29 - INFO - pyngrok.process.ngrok -   t=20

INFO:     116.206.247.149:0 - "POST / HTTP/1.1" 200 OK


...

05/29/2021 11:48:11 - INFO - pyngrok.process.ngrok -   t=2021-05-29T11:48:11+0000 lvl=info msg="join connections" obj=join id=02e1a33a3892 l=127.0.0.1:80 r=116.206.247.149:11125


INFO:     116.206.247.149:0 - "POST / HTTP/1.1" 200 OK


05/29/2021 11:49:15 - INFO - pyngrok.process.ngrok -   t=2021-05-29T11:49:15+0000 lvl=info msg="join connections" obj=join id=bb56ca9b110d l=127.0.0.1:80 r=116.206.247.149:11148


INFO:     116.206.247.149:0 - "POST / HTTP/1.1" 200 OK


Streaming output truncated to the last 5000 lines.
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7fc490e556e0> is already entered
05/29/2021 12:06:38 - ERROR - asyncio -   Exception in callback BaseAsyncIOLoop._handle_events(18, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(18, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7fc490e556e0> is already entered
05/29/2021 12:06:38 - ERROR - asyncio -   Exception in callback BaseAsyncIOLoop._handle_events(18, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(18, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7fc490e556e0> is already entered
05/29/2021 12: